# Chinese Audio Transcription with Whisper

This notebook transcribes Chinese audio from MP4 files using Whisper with MPS device support.

In [15]:
# %pip install python-dotenv

# import os
import torch
from dotenv import load_dotenv
# from pyannote.audio import Pipeline
from transformers import pipeline
import json

In [16]:
load_dotenv('../.env')
# Configuration
model_id = "openai/whisper-large-v3"
audio_file = "../audio/zhangzhiqiang.aac"

# Automatic device selection: CUDA for NVIDIA, else CPU (sorry AMD fans)
if torch.cuda.is_available():               # For ya boi Jensen Huang (NVIDIA)
    device = torch.device("cuda")
elif torch.backends.mps.is_available():     # For Apple Silicon (MPS)
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [13]:
pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v3",
    batch_size=32,
    torch_dtype = torch.float32,
    return_timestamps=True,
    generate_kwargs={
        "language": "Chinese",
        "task": "transcribe",
        "temperature": 0.0
    },
    device=device
)

/opt/miniconda3/envs/python-x64/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def transcribe(audio_path):
    """Transcribe audio file using the loaded Whisper pipeline"""
    try:
        print(f"Transcribing: {audio_path}")
        result = pipe(audio_path)
        return result
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

In [17]:
transcription = transcribe(audio_file)
print(transcription)

Transcribing: ../audio/zhangzhiqiang.aac
{'text': '我吃的鱼是鱼的我吃的鱼是鱼的你算玉族吗谁算啊那你可以采一家我的面包吗想吃玉芝菜了你这么漂亮还真是便宜我了呀那你给我买还真是便宜没好货你看得懂跳舞吗我跳过芭蕾跳过民族你跳过什么我跳过广告你长人感好吃吗好吃你吃出来吧 我还要吃呢你每天就吃你想吃的你跟我谈谈是为了什么为了第二个半价我可以直接听出来这是哪一块那你听听这是哪一块未付款你就买了一张电影票啊嗯 我先进去看了那我看什么那你把你眼睛给我你皱什么美啊你觉不觉得我脖子上面太白了怎么会呢不会呢 你看牙就挺好的我这个现眼花好看吗还挺好吧你要说好在哪儿耳水也不臭了你喝苹果汁吗你要喝吗你和我一个朋友一样我想喝啥都会给我谁啊自动三麦机我想试这个黑标餐巾20.24块痔疮基础 护理痔疮就不浅这个要240啊我的男朋友会给我买的吧你知道的我一直把你当亲妹妹对待着你追着干嘛恶犬出门都得牵着呀吃一个吧不行 我胃疼胃疼我教你 好听首先 这不是胃这是处于群音你怎么又想去旅游啊世界这么大我想去看看你盯着我干嘛你脸这么大我也想看看待这里都待腻了我是自由的我不属于任何地方我懂了你留着这次小长假想去哪玩去哪都行随便挑钱呢钱不是问题在这里有陈老板来安排我安排但是你买单不是说钱不是问题吗钱不是问题是答案这次假期我们订一个高级酒店再把什么按摩汤泉通通安排上如何你发财了说说你最近背着我干什么我最近在美团上发现了这个神券膨胀捧到了一张足疗买一送一还有这个酒店我刚刚也捧到了一张神券便宜了好多神券膨胀难怪最近12点18点21点都抱着手机那必须每天上美团上捧一捧就能领取多张大额神券还有不同额度的券随机捧出吃喝玩乐都省钱跟你说还有几率捧出免费的小吃黑金黑钻首次付费膨胀就能直接得小吃周六还有超大品牌限量直接get真假不信你上美团或者点击评论区链接每日捧一下 惊喜不断那你神圈捧账捧了多少翻倍捧的翻倍啊 那你多余的钱呢我看看还能捧出什么你快点啊我刚刚又捧账神圈捧到了一张说人SPA神圈还有景点门票神圈急什么 让我换完妆啊出去玩什么最重要性价比我觉得住好玩好 开心最重要那我不觉得我觉得出去最重要你都换一个小时了什么时候能出去什么时候能出去好了帮我选个口红就出门不是这都长得一样我怎么选啊你选个粉的就行和今天比较搭选个你喜欢的粉就行我喜欢的粉啊你早说啊我选好了不要钱买到的螺蛳粉啊刚用神

In [18]:
with open('transcription_zhangzhiqiang.json', 'w', encoding='utf-8') as f:
    json.dump(transcription, f, ensure_ascii=False, indent=2)

In [24]:
# Assuming transcription["chunks"] is a list of dicts like {"text": "..."}
output_path = "transcription_zhangzhiqiang.txt"

with open(output_path, "w", encoding="utf-8") as f:
    for chunk in transcription["chunks"]: # type: ignore
        f.write(chunk["text"].strip() + "\n") # type: ignore

print(f"✅ Saved {len(transcription['chunks'])} chunks to {output_path}") # type: ignore


✅ Saved 2372 chunks to transcription_zhangzhiqiang.txt
